In [1]:
import requests
import time
import pandas as pd
import numpy as np
import re
import json
import pdb
from bs4 import BeautifulSoup
import personal_keys
import ravelry_request_functions

In [4]:
def get_favs_list(username):
    fav_list = []
    favs_url = 'https://api.ravelry.com/people/{}/favorites/list.json'.format(username)
#     following request returns json with two keys: favorites and paginator
#     for the following request, I've added the params page size, 
#     which is 100 (maximum size), and page number (first page in the following request)
    favs = requests.get(favs_url, auth = (personal_keys.username(),personal_keys.password()),
                        params={'page_size':100, 'page':1})
#     the following will add a list of all the pattern ids for the favorited patterns and turns them into a list, which is then appended to the fav_list defined at the beginning of the function
    for item in range(0,len(favs.json()['favorites']:
        if 'pattern_id' in favs.json()['favorites'][item]['favorited'].keys():
            fav_list.append([favs.json()['favorites'][item]['favorited']['pattern_id'] 
                     for item in range(0,len(favs.json()['favorites'])) 
                     if favs.json()['favorites'][item]['favorited'] is not None])
        else:
            fav_list.append([favs.json()['favorites'][item]['favorited']['id'] 
                     for item in range(0,len(favs.json()['favorites'])) 
                     if favs.json()['favorites'][item]['favorited'] is not None])

#     in the event the user has more than 100 favorites, 
# this loop will essentially go in and make a new request for the next page of likes, 
# this loop finishes when the page number (which starts at 2 because we've already requested page one above) 
# equals the last page
    if favs.json()['paginator']['page_count']>1:
#         page_number is 2 because first page is already in fav_list
        page_number = 2
#         loop to do successive requests for new pages of favorited pattern ids
        while page_number <= favs.json()['paginator']['last_page']:
            new_request_favs = requests.get(favs_url, auth = (personal_keys.username(),personal_keys.password()),
                        params={'page_size':100, 'page':page_number})
#             append the new list of favorites to the old list containing the first 100 favorites
            fav_list.append([new_request_favs.json()['favorites'][item]['favorited']['id'] 
                             for item in range(0,len(new_request_favs.json()['favorites'])) 
                             if new_request_favs.json()['favorites'][item]['favorited'] is not None])
#             add one to the page number so it will request the page following page 2, this will be done for as long as there are more pages to be requested
            page_number += 1
#     since the fav_list is currently a list of lists, the following function flattens the fav_list into one long list of ids
    flat_list = [item for sublist in fav_list for item in sublist]
    return flat_list

In [2]:
kate_fav_list = ravelry_request_functions.get_favs_list('katec125')
edited_kate_fav_list = ravelry_request_functions.problem_children(kate_fav_list)
taken_out_favs = [p for p in kate_fav_list if str(p) not in edited_kate_fav_list]



In [3]:
favs_url = 'https://api.ravelry.com/people/{}/favorites/list.json'.format('katec125')

favs = requests.get(favs_url, auth = (personal_keys.username(),personal_keys.password()),params={'page_size':100, 'page':1})

bad_favs_list =[favs.json()['favorites'][item]['favorited'] for item in range(0,len(favs.json()['favorites'])) if favs.json()['favorites'][item]['favorited']['id'] in taken_out_favs]

In [8]:
bad_favs_list[5]['pattern_id']

497647

In [13]:
good_favs_list = [favs.json()['favorites'][item]['favorited'] for item in range(0,len(favs.json()['favorites'])) if favs.json()['favorites'][item]['favorited']['id'] not in taken_out_favs]

good_favs_list[5].keys()

dict_keys(['id', 'name', 'permalink', 'first_photo', 'designer', 'pattern_author', 'pattern_sources'])

In [10]:
patterns_url = 'https://api.ravelry.com/patterns/497647.json'
patterns = requests.get(patterns_url, auth = (personal_keys.username(),personal_keys.password()))
patterns.json()

.com/uploads/thepluckyknitter/239054377/Dotted_Rays_3_medium2.jpg',
    'small2_url': 'https://images4-f.ravelrycache.com/uploads/thepluckyknitter/239054377/Dotted_Rays_3_small2.jpg',
    'caption': None,
    'caption_html': None,
    'copyright_holder': 'thepluckyknitter'},
   {'id': 48104301,
    'sort_order': 9,
    'x_offset': 0,
    'y_offset': 0,
    'square_url': 'https://images4-g.ravelrycache.com/uploads/putyourflareon/289664144/image_square.jpg',
    'medium_url': 'https://images4-g.ravelrycache.com/uploads/putyourflareon/289664144/image_medium.jpg',
    'thumbnail_url': 'https://images4-g.ravelrycache.com/uploads/putyourflareon/289664144/image_thumbnail.jpg',
    'small_url': 'https://images4-g.ravelrycache.com/uploads/putyourflareon/289664144/image_small_best_fit.jpg',
    'flickr_url': None,
    'shelved_url': 'https://images4-f.ravelrycache.com/uploads/putyourflareon/289664144/image_shelved.jpg',
    'medium2_url': 'https://images4-f.ravelrycache.com/uploads/putyourflareo